# Build Tech tree representation
Natalia Vélez, July 2020

In [7]:
%matplotlib inline

import os, re, glob

import numpy as np
import pandas as pd
from os.path import join as opj

import ohol_categories as cat
import ohol_objects as obj
import ohol_transitions as trans

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from tqdm import notebook

{'id': 513,
 'name': 'Flooded Pond',
 'containable': 0,
 'containSize': 1,
 'vertSlotRot': 0.0,
 'permanent': 1,
 'minPickupAge': 3,
 'heldInHand': 0,
 'blocksWalking': 1,
 'leftBlockingRadius': 0,
 'rightBlockingRadius': 0,
 'drawBehindPlayer': 0,
 'mapChance': 0.0,
 'heatValue': 0,
 'rValue': 0.0,
 'person': 0,
 'noSpawn': 0,
 'male': 0,
 'deathMarker': 0,
 'homeMarker': 0,
 'floor': 0,
 'floorHugging': 0,
 'foodValue': 0,
 'speedMult': 1.0,
 'heldOffset': [0.0, 0.0],
 'clothing': 'n',
 'clothingOffset': [0.0, 0.0],
 'deadlyDistance': 0,
 'useDistance': 1,
 'sounds': ['-1:0.0', '429:0.250000', '-1:0.0', '-1:0.0'],
 'creationSoundInitialOnly': 0,
 'numSlots': 0,
 'timeStretch': 1.0,
 'slotSize': 1,
 'numSprites': 4,
 'spriteID': 411,
 'pos': [6.0, 16.0],
 'rot': 1.0,
 'hFlip': 0,
 'color': [1.0, 1.0, 1.0],
 'ageRange': [-1.0, -1.0],
 'parent': -1,
 'invisHolding': 0,
 'invisWorn': 0,
 'behindSlots': 0,
 'headIndex': -1,
 'bodyIndex': -1,
 'backFootIndex': -1,
 'frontFootIndex': -1,
 '

## Find files

Find transition files:

In [4]:
gsearch = lambda *args: glob.glob(opj(*args))

data_dir = '../../OneLifeData7/'
trans_dir  = opj(data_dir, 'transitions')
trans_files = gsearch(trans_dir, '*.txt')

print(*trans_files[:5], sep='\n')

../../OneLifeData7/transitions/-1_2574.txt
../../OneLifeData7/transitions/0_702.txt
../../OneLifeData7/transitions/314_235.txt
../../OneLifeData7/transitions/2165_2165.txt
../../OneLifeData7/transitions/0_1692.txt


In [5]:
trans.read_transition(opj(trans_dir, '59_2226.txt'))

{'origActor': 59,
 'origTarget': 2226,
 'newActor': 0,
 'newTarget': 2220,
 'autoDecaySeconds': 0,
 'actorMinUseFraction': 0.0,
 'targetMinUseFraction': 0.0,
 'reverseUseActor': 0,
 'reverseUseTarget': 0,
 'move': 0,
 'desiredMoveDist': 1,
 'noUseActor': 0,
 'noUseTarget': 0,
 'lastUseActor': False,
 'lastUseTarget': False,
 'origActorName': 'Rope',
 'origTargetName': 'Newcomen Pump without Rope',
 'newActorName': 'Empty',
 'newTargetName': 'Dry Newcomen Pump',
 'isTool': True}

In [40]:
cat.cat_children(2220)

[2240, 2270, 2280, 2314, 2305, 2312, 2359, 2500]

In [45]:
obj.read_obj(1487)

{'id': 1487,
 'name': 'Dead Domestic Calf',
 'containable': 0,
 'containSize': 1.0,
 'vertSlotRot': 0.0,
 'permanent': 0,
 'minPickupAge': 3,
 'heldInHand': 0,
 'blocksWalking': 0,
 'leftBlockingRadius': 0,
 'rightBlockingRadius': 0,
 'drawBehindPlayer': 0,
 'mapChance': 0.0,
 'heatValue': 0,
 'rValue': 0.0,
 'person': 0,
 'noSpawn': 0,
 'male': 0,
 'deathMarker': 0,
 'homeMarker': 0,
 'floor': 0,
 'floorHugging': 0,
 'foodValue': 0,
 'speedMult': 1.0,
 'heldOffset': [8.0, 19.0],
 'clothing': 'n',
 'clothingOffset': [0.0, 0.0],
 'deadlyDistance': 0,
 'useDistance': 1,
 'sounds': ['1242:0.150000', '-1:0.0', '-1:0.0', '-1:0.0'],
 'creationSoundInitialOnly': 0,
 'numSlots': 0,
 'timeStretch': 1.0,
 'slotSize': 1.0,
 'slotsLocked': 0,
 'numSprites': 8,
 'spriteID': 1630,
 'pos': [25.0, -18.0],
 'rot': 0.25,
 'hFlip': 0,
 'color': [1.0, 1.0, 1.0],
 'ageRange': [-1.0, -1.0],
 'parent': 6,
 'invisHolding': 0,
 'invisWorn': 0,
 'behindSlots': 0,
 'headIndex': -1,
 'bodyIndex': -1,
 'backFootIn

Object files:

In [10]:
obj_dir = opj(data_dir, 'objects')
obj_files = gsearch(obj_dir, '*txt')

print(*obj_files[:5], sep='\n')

../../OneLifeData7/objects/3644.txt
../../OneLifeData7/objects/1053.txt
../../OneLifeData7/objects/1735.txt
../../OneLifeData7/objects/3122.txt
../../OneLifeData7/objects/2228.txt


## Parse objects
Build a dictionary of objectID-mapChance pairs (to identify natural objects)

In [11]:
str_extract = lambda pattern, s: re.search(pattern, s).group(0)
int_extract = lambda pattern, s: int(str_extract(pattern, s))

obj_dict = {}
for o in obj_files:
    is_obj = re.search('nextObjectNumber|groundHeat', o) is None
    
    if is_obj:
        o_num = int_extract('[0-9]+(?=.txt)', o)

        o_data = obj.read_obj(o_num)
        obj_dict[o_num] = o_data['mapChance']

Natural objects:

In [15]:
natural_objs = [o for (o,p) in obj_dict.items() if p > 0]
natural_objs.sort()
print(*natural_objs[:20], sep='\t')

30	32	33	36	49	50	63	65	99	100	121	125	133	136	153	161	211	242	291	418


Let's check these to make sure we're gettting reasonable objects:

In [1]:
obj.read_obj(513)

NameError: name 'obj' is not defined

In [21]:
for o in natural_objs:
    print('%i: %s' % (o, obj.obj_name(o)))

30: Wild Gooseberry Bush
32: Big Hard Rock
33: Stone
36: Seeding Wild Carrot
49: Juniper Tree
50: Milkweed
63: Maple Tree -Branch
65: Lombardy Poplar Tree -Branch
99: White Pine Tree
100: White Pine Tree with Needles
121: Tule Reeds
125: Clay Deposit
133: Flint
136: Sapling
153: Yew Tree -Branch
161: Rabbit Hole -hiding &single
211: Fertile Soil Deposit
242: Ripe Wheat
291: Flat Rock - empty
418: Wolf
527: Willow Tree
530: Bald Cypress Tree
531: Mouflon
630: Bear Cave
674: Limestone
680: Gold Vein
703: Penguin
706: Ice Hole
707: Antarctic Fur Seal
713: Indigo
714: Rose Madder
729: Alum
760: Dead Tree
761: Barrel Cactus
764: Rattle Snake
769: Wild Horse
791: Monolith
804: Burdock
805: Wild Onion
942: Muddy Iron Vein - gridPlacement40 &40
1013: Wild Rose with Fruit
1020: Snow Bank
1107: Teosinte
1140: Wild Potato
1157: Wild Bean Plant
1184: Wild Squash Plant
1203: Wild Cabbage
1261: Canada Goose Pond with Egg
1323: Wild Boar
1435: Bison
1874: Wild Mango Tree
1890: Lapis Lazuli
1891: Cinn

## Parse transitions

Dataframe of transitions:

In [34]:
trans_cats

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

'FFFF'

In [40]:
np.unique(trans_cat_list, return_counts = True)

(array(['[False, False, False, False]', '[False, False, False, True]',
        '[False, False, True, False]', '[False, False, True, True]',
        '[False, True, False, False]', '[False, True, False, True]',
        '[False, True, True, False]', '[False, True, True, True]',
        '[True, False, False, False]', '[True, False, False, True]',
        '[True, False, True, False]', '[True, False, True, True]',
        '[True, True, False, False]', '[True, True, False, True]',
        '[True, True, True, False]', '[True, True, True, True]'],
       dtype='<U28'),
 array([3746,   20,   76,  145,    7,    1,   25,    6,   55,   38,    7,
           7,  183,    6,    6,   13]))

In [48]:
trans_keys = ['origActor', 'newActor', 'origTarget', 'newTarget']
trans_list = []
for f in trans_files:
    trans_dict = trans.read_transition(f)
    trans_objs = [trans_dict[k] for k in trans_keys]
    
    # Are any of these categories?
    trans_cats = [cat.is_cat(o) for o in trans_objs]
    trans_cat_str = ''.join(re.findall('[A-Z]', str(trans_cats)))
    trans_dict['isCat'] = trans_cat_str
    
    trans_list.append(trans_dict)

Assemble into dataframe:

In [49]:
trans_df = pd.DataFrame(trans_list)
trans_df['isDecay'] = trans_df['autoDecaySeconds'] > 0
trans_df = trans_df[['origActor', 'origTarget', 'newActor', 'newTarget', 'isDecay', 'isTool', 'isCat']]

trans_df.head()

,origActor,origTarget,newActor,newTarget,isDecay,isTool,isCat
0,-1,2574,0,2578,True,False,FFFF
1,0,702,425,695,False,False,FFFF
2,314,235,0,317,False,True,FFFF
3,2165,2165,235,3699,False,True,FFFF
4,0,1692,1719,1706,False,False,FFFF


Count each type of category transition:

In [67]:
cat_counts = trans_df.groupby('isCat')['isTool'].agg('count').reset_index()
cat_counts = cat_counts.rename(columns = {'isTool': 'n'})

,isCat,n
0,FFFF,3746
1,FFFT,20
2,FFTF,76
3,FFTT,145
4,FTFF,7
5,FTFT,1
6,FTTF,25
7,FTTT,6
8,TFFF,55
9,TFFT,38


Check examples of category transitions:

In [87]:
cat_trans = trans_df.groupby('isCat').first().reset_index()
cat_trans = pd.merge(cat_counts, cat_trans, on = 'isCat')
cat_trans = cat_trans.sort_values('n', ascending = False).reset_index(drop=True)
cat_trans

,isCat,n,origActor,origTarget,newActor,newTarget,isDecay,isTool
0,FFFF,3746,-1,2574,0,2578,True,False
1,TTFF,183,1127,2883,1127,2879,False,True
2,FFTT,145,516,519,520,519,False,False
3,FFTF,76,209,412,210,512,False,False
4,TFFF,55,853,3374,852,1465,False,False
5,TFFT,38,324,-1,239,322,False,True
6,FTTF,25,0,1069,969,1012,False,False
7,FFFT,20,812,665,0,2233,False,True
8,TTTT,13,394,1802,394,1806,False,True
9,FTFF,7,568,63,1829,63,False,False


In [231]:
cat.read_cat(1127)

[1126, 192]

In [234]:
trans.read_transition(opj(trans_dir, '209_412_LT.txt'))

{'origActor': 209,
 'origTarget': 412,
 'newActor': 210,
 'newTarget': 512,
 'autoDecaySeconds': 0,
 'actorMinUseFraction': 0.0,
 'targetMinUseFraction': 0.0,
 'reverseUseActor': 0,
 'reverseUseTarget': 0,
 'move': 0,
 'desiredMoveDist': 1,
 'lastUseActor': False,
 'lastUseTarget': True,
 'origActorName': 'Empty Water Pouch',
 'origTargetName': '@ Wet Canada Goose Pond',
 'newActorName': 'Full Water Pouch',
 'newTargetName': 'Dry Pond',
 'isTool': False}

Look into these examples:

In [235]:
cat.read_cat(969)

[956, 951, 955, 1073, 973, 957]

In [236]:
obj.read_obj(957)

{'id': 957,
 'name': 'Letter D',
 'containable': 1,
 'containSize': 0.5,
 'vertSlotRot': -0.25,
 'permanent': 0,
 'minPickupAge': 3,
 'heldInHand': 1,
 'blocksWalking': 0,
 'leftBlockingRadius': 0,
 'rightBlockingRadius': 0,
 'drawBehindPlayer': 0,
 'mapChance': 0.0,
 'heatValue': 0,
 'rValue': 0.0,
 'person': 0,
 'noSpawn': 0,
 'male': 0,
 'deathMarker': 0,
 'homeMarker': 0,
 'floor': 0,
 'floorHugging': 0,
 'foodValue': 0,
 'speedMult': 1.0,
 'heldOffset': [4.0, -9.0],
 'clothing': 'n',
 'clothingOffset': [0.0, 0.0],
 'deadlyDistance': 0,
 'useDistance': 1,
 'sounds': ['159:0.250000', '-1:0.0', '-1:0.0', '-1:0.0'],
 'creationSoundInitialOnly': 1,
 'numSlots': 0,
 'timeStretch': 1.0,
 'slotSize': 1.0,
 'slotsLocked': 0,
 'numSprites': 1,
 'spriteID': 1209,
 'pos': [3.0, -34.0],
 'rot': 0.0,
 'hFlip': 0,
 'color': [1.0, 1.0, 1.0],
 'ageRange': [-1.0, -1.0],
 'parent': -1,
 'invisHolding': 0,
 'invisWorn': 0,
 'behindSlots': 0,
 'headIndex': -1,
 'bodyIndex': -1,
 'backFootIndex': -1,
 

## Parse categories

## Build up Tech tree